In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def model():
    model = keras.Sequential([
        layers.Dense(2, activation="relu", name="input"),
        layers.Dense(3, activation="relu"),
        layers.Dense(48, name="output"),
    ])

In [20]:
# every possible move the AI can make
def right():
    controller.tilt_analog(melee.Button.BUTTON_MAIN,1,.5)
    console.step()
    controller.release_all()
def left():
    controller.tilt_analog(melee.Button.BUTTON_MAIN,0,.5)
    console.step()
    controller.release_all()
def jump():
    controller.press_button(melee.Button.BUTTON_X)
    console.step()
    console.step()
    console.step()
    controller.release_all()
def shorthop():
    if gamestate.player[2].jumps_left == 2 and gamestate.player[2].action != melee.Action.KNEE_BEND:
                controller.press_button(melee.Button.BUTTON_X)
                console.step()
                controller.release_all()
                console.step()
                console.step()

In [18]:
#model
def nextAction(gamestate):
    inputarr = [gamestate.player[1].x,gamestate.player[1].y,gamestate.player[2].x,gamestate.player[2].y]
    prediction = model.predict(inputarr)
    if prediction == 0:
        return right()# right,
    elif prediction == 1: 
        return left()# left,
    elif prediction ==2: 
        return # jump,
    elif prediction == 3: 
        return # short hop .5,
    elif prediction == 4: 
        return # short hop forward 1,
    elif prediction == 5: 
        return # short hop forward .75,
    elif prediction == 6: 
        return # short hop backward .25,
    elif prediction == 7: 
        return # short hop backward 0,
    elif prediction == 8: 
        return # standing grab,
    elif prediction == 9: 
        return # jump cancel grab,
    elif prediction == 10: 
        return # spot dodge,
    elif prediction == 11: 
        return # shield,
    elif prediction == 12: 
        return # wave dash left 0,
    elif prediction == 13: 
        return # wave dash left .25,
    elif prediction == 14: 
        return # wave dash right 1,
    elif prediction == 15: 
        return # wave dash right .75,
    elif prediction == 16: 
        return # wave dash in-place,
    elif prediction == 17: 
        return # up air,
    elif prediction == 18: 
        return # down air,
    elif prediction == 19: 
        return # forward air, 
    elif prediction == 20: 
        return # back air,
    elif prediction == 21: 
        return # neutral air,
    elif prediction == 22: 
        return #neutral b,
    elif prediction == 23: 
        return #side b,
    elif prediction == 24: 
        return #down b,
    elif prediction == 25: 
        return #up b,
    elif prediction == 26: 
        return # amsa teq left,
    elif prediction == 27: 
        return # amsa teq right,
    elif prediction == 28: 
        return # amsa teq neutral,
    elif prediction == 29: 
        return # slide off di,
    elif prediction == 30: 
        return # di down away,
    elif prediction == 31: 
        return # survival di,
    elif prediction == 32: 
        return # combo di,
    elif prediction == 33: 
        return #fast fall,
    elif prediction == 34: 
        return #jab,
    elif prediction == 35: 
        return #forward tilt,
    elif prediction == 36: 
        return #down tilt,
    elif prediction == 37: 
        return #up tilt,
    elif prediction == 38: 
        return # dash right 1,
    elif prediction == 39: 
        return # dash right .75,
    elif prediction == 40: 
        return # dash right .25,
    elif prediction == 41: 
        return # dash left 1,
    elif prediction == 42: 
        return # dash left .75,
    elif prediction == 43: 
        return # dash left .25,
    elif prediction == 44: 
        return # pivot,
    elif prediction == 45: 
        return # crouch,
    elif prediction == 46: 
        return # c stick down
    elif prediction == 47:
        return #head empty; no thoughts

In [2]:
#model interface

import argparse
import signal
import sys
import melee
# This example program demonstrates how to use the Melee API to run a console,
#   setup controllers, and send button presses over to a console (dolphin or Slippi/Wii)

def check_port(value):
    ivalue = int(value)
    if ivalue < 1 or ivalue > 4:
        raise argparse.ArgumentTypeError("%s is an invalid controller port. \
                                         Must be 1, 2, 3, or 4." % value)
    return ivalue

parser = argparse.ArgumentParser(description='Example of libmelee in action')
parser.add_argument('--port', '-p', type=check_port,
                    help='The controller port (1-4) your AI will play on',
                    default=2)
parser.add_argument('--opponent', '-o', type=check_port,
                    help='The controller port (1-4) the opponent will play on',
                    default=1)
parser.add_argument('--debug', '-d', action='store_true',
                    help='Debug mode. Creates a CSV of all game states')
parser.add_argument('--framerecord', '-r', default=False, action='store_true',
                    help='(DEVELOPMENT ONLY) Records frame data from the match,' \
                    'stores into framedata.csv.')
parser.add_argument('--address', '-a', default="127.0.0.1",
                    help='IP address of Slippi/Wii')
parser.add_argument('--dolphin_executable_path', '-e', default="D:/dmarq/Downloads/Ishiiruka-gamma/Binary/x64",
                    help='Manually specify the non-installed directory where dolphin is')
parser.add_argument('--connect_code', '-t', default="",
                    help='Direct connect code to connect to in Slippi Online')

args = parser.parse_args()

# This logger object is useful for retroactively debugging issues in your bot
#   You can write things to it each frame, and it will create a CSV file describing the match
log = None
if args.debug:
    log = melee.Logger()

# This frame data object contains lots of helper functions and values for looking up
#   various Melee stats, hitboxes, and physics calculations
framedata = melee.FrameData(args.framerecord)

# Create our Console object.
#   This will be one of the primary objects that we will interface with.
#   The Console represents the virtual or hardware system Melee is playing on.
#   Through this object, we can get "GameState" objects per-frame so that your
#       bot can actually "see" what's happening in the game
console = melee.Console(path=args.dolphin_executable_path,
                        slippi_address=args.address,
                        slippi_port=51441,
                        blocking_input=False,
                        logger=log)

# Dolphin has an optional mode to not render the game's visuals
#   This is useful for BotvBot matches
console.render = True

# Create our Controller object
#   The controller is the second primary object your bot will interact with
#   Your controller is your way of sending button presses to the game, whether
#   virtual or physical.
controller = melee.Controller(console=console,
                              port=args.port,
                              type=melee.ControllerType.STANDARD)

controller_opponent = melee.Controller(console=console,
                                       port=args.opponent,
                                       type=melee.ControllerType.GCN_ADAPTER)

# This isn't necessary, but makes it so that Dolphin will get killed when you ^C
def signal_handler(sig, frame):
    console.stop()
    if args.debug:
        log.writelog()
        print("") #because the ^C will be on the terminal
        print("Log file created: " + log.filename)
    print("Shutting down cleanly...")
    if args.framerecord:
        framedata.save_recording()
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

# Run the console
console.run()

# Connect to the console
print("Connecting to console...")
if not console.connect():
    print("ERROR: Failed to connect to the console.")
    print("\tIf you're trying to autodiscover, local firewall settings can " +
          "get in the way. Try specifying the address manually.")
    sys.exit(-1)

# Plug our controller in
#   Due to how named pipes work, this has to come AFTER running dolphin
#   NOTE: If you're loading a movie file, don't connect the controller,
#   dolphin will hang waiting for input and never receive it
print("Connecting controller to console...")
if not controller.connect():
    print("ERROR: Failed to connect the controller.")
    sys.exit(-1)
print("Controller connected")

# Main loop
while True:
    # "step" to the next frame
    gamestate = console.step()

    # The console object keeps track of how long your bot is taking to process frames
    #   And can warn you if it's taking too long
    if console.processingtime * 1000 > 12:
        print("WARNING: Last frame took " + str(console.processingtime*1000) + "ms to process.")

    # What menu are we in?
    if gamestate.menu_state in [melee.Menu.IN_GAME, melee.Menu.SUDDEN_DEATH]:

        # Slippi Online matches assign you a random port once you're in game that's different
        #   than the one you're physically plugged into. This helper will autodiscover what
        #   port we actually are.
        discovered_port = melee.gamestate.port_detector(gamestate, controller, melee.Character.FOX)

        if discovered_port > 0:
            if args.framerecord:
                framedata._record_frame(gamestate)
                
            # NOTE: This is where your AI does all of its stuff!
            # This line will get hit once per frame, so here is where you read
            #   in the gamestate and decide what buttons to push on the controller
            
            
            
            controller.tilt_analog(melee.Button.BUTTON_MAIN, 0, .5)
            controller.press_button(melee.Button.BUTTON_A)
            console.step()
            controller.release_button(melee.Button.BUTTON_A)
            
    else:
        melee.MenuHelper.menu_helper_simple(gamestate,
                                            controller,
                                            args.port,
                                            melee.Character.GAMEANDWATCH,
                                            melee.Stage.FINAL_DESTINATION,
                                            args.connect_code,
                                            autostart=False,
                                            swag=False)
    if log:
        log.logframe(gamestate)
        log.writeframe()


usage: ipykernel_launcher.py [-h] [--port PORT] [--opponent OPPONENT]
                             [--debug] [--framerecord] [--address ADDRESS]
                             [--dolphin_executable_path DOLPHIN_EXECUTABLE_PATH]
                             [--connect_code CONNECT_CODE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\dmarq\AppData\Roaming\jupyter\runtime\kernel-1dd5d2f7-530b-4a34-9d2e-83d716e3e9e1.json


SystemExit: 2

d:\dmarq\tools\python37\lib\site-packages\IPython\core\interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
